In [1]:
import folium

import pandas.io.sql as psql
import psycopg2 as pg

import random

/Users/ca/PycharmProjects/JupyterNoteBookExample/venv/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
with pg.connect(database='chi-navi',
                host='localhost',
                user='postgres',
                port=5432) as conn:
    bus_stop_route_sql = \
        'with route_bus_stop_list as( ' \
        'select ' \
        'r.route_id, b.bus_stop_id,rbs.ordinal ' \
        'from ' \
        'bus_stop b,route r,route_bus_stop rbs ' \
        'where ' \
        'b.bus_stop_id = rbs.bus_stop_id ' \
        'and r.route_id = rbs.route_id ' \
        'order by r.route_id,rbs.ordinal' \
        '), ' \
        'task_bus_stop_list as ( ' \
        'select ' \
        'distinct td.bus_stop_id ' \
        'from ' \
        'task_delay td ' \
        ') ' \
        'select ' \
        'distinct t.bus_stop_id, r.route_id ' \
        'from ' \
        'task_bus_stop_list t, route_bus_stop_list r ' \
        'where ' \
        't.bus_stop_id = r.bus_stop_id ' \
        'order by t.bus_stop_id, r.route_id'


In [3]:
with pg.connect(database='chi-navi',
                host='localhost',
                user='postgres',
                port=5432) as conn:
    bus_stop_sql = \
        'select ' \
        'distinct b.bus_stop_id, latlon[1] as lat, latlon[0] as lon ' \
        'from ' \
        '(select bus_stop_id,latlon::point as latlon from bus_stop ) b, task_delay t ' \
        'where ' \
        'b.bus_stop_id = t.bus_stop_id ' \
        'order by bus_stop_id'        

In [4]:
with pg.connect(database='chi-navi',
                host='localhost',
                user='postgres',
                port=5432) as conn:
    bus_course_sql = \
        'with tmp as (' \
        'select ' \
        'route_id, latlon::point as latlon, ordinal ' \
        'from ' \
        'bus_course ' \
        ')' \
        'select ' \
        'route_id, latlon[1] as lat, latlon[0] as lon, ordinal ' \
        'from ' \
        'tmp ' \
        'order by route_id, ordinal'


In [5]:
bus_stop_route_sql_result = psql.read_sql(bus_stop_route_sql, conn)
bus_course_sql_result = psql.read_sql(bus_course_sql, conn)
bus_stop_sql_result = psql.read_sql(bus_stop_sql, conn)



In [10]:
def create_bus_stop_course(bus_stop_id):
    bus_stop_data = bus_stop_sql_result[bus_stop_sql_result['bus_stop_id'] == bus_stop_id]
    route_id_list = bus_stop_route_sql_result[bus_stop_route_sql_result['bus_stop_id'] == bus_stop_id]['route_id']
    course_list = bus_course_sql_result[bus_course_sql_result['route_id'].isin(route_id_list)]

    return course_list, route_id_list, bus_stop_data


In [11]:
course_list, route_id_list, bus_stop_data = create_bus_stop_course(182)

In [12]:
def generate_random_color_code():
    return "rgb({},{},{})".format(random.randint(10, 255), random.randint(10, 255), random.randint(10, 255))


In [13]:
chitose_location = [42.82820516925404, 141.65187835693362]

osm_map = folium.Map(location=chitose_location, zoom_start=14)

for i, route_id in enumerate(route_id_list):
    feature_group = folium.FeatureGroup(name='route_id:{}'.format(route_id))
    course_latlon_list = course_list[course_list['route_id'] == route_id][['lat', 'lon']].values.tolist()
    feature_group.add_child(folium.PolyLine(
        locations=[course_latlon_list],
        color=generate_random_color_code(),
        )
    )
    osm_map.add_child(feature_group)

# marker_feature_group = folium.FeatureGroup(name='bus_stop_id')
# marker_feature_group.add_child(folium.Marker([bus_stop_data['lat'], bus_stop_data['lon']]))
# 
# osm_map.add_child(marker_feature_group)

osm_map.add_child(folium.LayerControl())


<div style="width:100%;"><div style="position:relative;width:100%;height:0;padding-bottom:60%;"><iframe src="data:text/html;charset=utf-8;base64,PCFET0NUWVBFIGh0bWw+CjxoZWFkPiAgICAKICAgIDxtZXRhIGh0dHAtZXF1aXY9ImNvbnRlbnQtdHlwZSIgY29udGVudD0idGV4dC9odG1sOyBjaGFyc2V0PVVURi04IiAvPgogICAgPHNjcmlwdD5MX1BSRUZFUl9DQU5WQVM9ZmFsc2U7IExfTk9fVE9VQ0g9ZmFsc2U7IExfRElTQUJMRV8zRD1mYWxzZTs8L3NjcmlwdD4KICAgIDxzY3JpcHQgc3JjPSJodHRwczovL2Nkbi5qc2RlbGl2ci5uZXQvbnBtL2xlYWZsZXRAMS4yLjAvZGlzdC9sZWFmbGV0LmpzIj48L3NjcmlwdD4KICAgIDxzY3JpcHQgc3JjPSJodHRwczovL2FqYXguZ29vZ2xlYXBpcy5jb20vYWpheC9saWJzL2pxdWVyeS8xLjExLjEvanF1ZXJ5Lm1pbi5qcyI+PC9zY3JpcHQ+CiAgICA8c2NyaXB0IHNyYz0iaHR0cHM6Ly9tYXhjZG4uYm9vdHN0cmFwY2RuLmNvbS9ib290c3RyYXAvMy4yLjAvanMvYm9vdHN0cmFwLm1pbi5qcyI+PC9zY3JpcHQ+CiAgICA8c2NyaXB0IHNyYz0iaHR0cHM6Ly9jZG5qcy5jbG91ZGZsYXJlLmNvbS9hamF4L2xpYnMvTGVhZmxldC5hd2Vzb21lLW1hcmtlcnMvMi4wLjIvbGVhZmxldC5hd2Vzb21lLW1hcmtlcnMuanMiPjwvc2NyaXB0PgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL2Nkbi5qc2RlbGl2ci5uZXQvbnBtL2xlYWZsZXRAMS4yLjAvZGlzdC9sZWFmbGV0LmNzcyIvPgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL21heGNkbi5ib290c3RyYXBjZG4uY29tL2Jvb3RzdHJhcC8zLjIuMC9jc3MvYm9vdHN0cmFwLm1pbi5jc3MiLz4KICAgIDxsaW5rIHJlbD0ic3R5bGVzaGVldCIgaHJlZj0iaHR0cHM6Ly9tYXhjZG4uYm9vdHN0cmFwY2RuLmNvbS9ib290c3RyYXAvMy4yLjAvY3NzL2Jvb3RzdHJhcC10aGVtZS5taW4uY3NzIi8+CiAgICA8bGluayByZWw9InN0eWxlc2hlZXQiIGhyZWY9Imh0dHBzOi8vbWF4Y2RuLmJvb3RzdHJhcGNkbi5jb20vZm9udC1hd2Vzb21lLzQuNi4zL2Nzcy9mb250LWF3ZXNvbWUubWluLmNzcyIvPgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL2NkbmpzLmNsb3VkZmxhcmUuY29tL2FqYXgvbGlicy9MZWFmbGV0LmF3ZXNvbWUtbWFya2Vycy8yLjAuMi9sZWFmbGV0LmF3ZXNvbWUtbWFya2Vycy5jc3MiLz4KICAgIDxsaW5rIHJlbD0ic3R5bGVzaGVldCIgaHJlZj0iaHR0cHM6Ly9yYXdnaXQuY29tL3B5dGhvbi12aXN1YWxpemF0aW9uL2ZvbGl1bS9tYXN0ZXIvZm9saXVtL3RlbXBsYXRlcy9sZWFmbGV0LmF3ZXNvbWUucm90YXRlLmNzcyIvPgogICAgPHN0eWxlPmh0bWwsIGJvZHkge3dpZHRoOiAxMDAlO2hlaWdodDogMTAwJTttYXJnaW46IDA7cGFkZGluZzogMDt9PC9zdHlsZT4KICAgIDxzdHlsZT4jbWFwIHtwb3NpdGlvbjphYnNvbHV0ZTt0b3A6MDtib3R0b206MDtyaWdodDowO2xlZnQ6MDt9PC9zdHlsZT4KICAgIAogICAgPHN0eWxlPiNtYXBfMjM2MTI0MTlmMTMyNDM3NGExOTQ2NDgxN2FlNTZkYmEgewogICAgICAgIHBvc2l0aW9uOiByZWxhdGl2ZTsKICAgICAgICB3aWR0aDogMTAwLjAlOwogICAgICAgIGhlaWdodDogMTAwLjAlOwogICAgICAgIGxlZnQ6IDAuMCU7CiAgICAgICAgdG9wOiAwLjAlOwogICAgICAgIH0KICAgIDwvc3R5bGU+CjwvaGVhZD4KPGJvZHk+ICAgIAogICAgCiAgICA8ZGl2IGNsYXNzPSJmb2xpdW0tbWFwIiBpZD0ibWFwXzIzNjEyNDE5ZjEzMjQzNzRhMTk0NjQ4MTdhZTU2ZGJhIiA+PC9kaXY+CjwvYm9keT4KPHNjcmlwdD4gICAgCiAgICAKICAgIAogICAgICAgIHZhciBib3VuZHMgPSBudWxsOwogICAgCgogICAgdmFyIG1hcF8yMzYxMjQxOWYxMzI0Mzc0YTE5NDY0ODE3YWU1NmRiYSA9IEwubWFwKAogICAgICAgICdtYXBfMjM2MTI0MTlmMTMyNDM3NGExOTQ2NDgxN2FlNTZkYmEnLCB7CiAgICAgICAgY2VudGVyOiBbNDIuODI4MjA1MTY5MjU0MDQsIDE0MS42NTE4NzgzNTY5MzM2Ml0sCiAgICAgICAgem9vbTogMTQsCiAgICAgICAgbWF4Qm91bmRzOiBib3VuZHMsCiAgICAgICAgbGF5ZXJzOiBbXSwKICAgICAgICB3b3JsZENvcHlKdW1wOiBmYWxzZSwKICAgICAgICBjcnM6IEwuQ1JTLkVQU0czODU3LAogICAgICAgIHpvb21Db250cm9sOiB0cnVlLAogICAgICAgIH0pOwoKICAgIAogICAgCiAgICB2YXIgdGlsZV9sYXllcl9jZTFiNjZmNmE1ODA0NDQ3YjMxYzRjNmE5NmJiMWNjZiA9IEwudGlsZUxheWVyKAogICAgICAgICdodHRwczovL3tzfS50aWxlLm9wZW5zdHJlZXRtYXAub3JnL3t6fS97eH0ve3l9LnBuZycsCiAgICAgICAgewogICAgICAgICJhdHRyaWJ1dGlvbiI6IG51bGwsCiAgICAgICAgImRldGVjdFJldGluYSI6IGZhbHNlLAogICAgICAgICJtYXhOYXRpdmVab29tIjogMTgsCiAgICAgICAgIm1heFpvb20iOiAxOCwKICAgICAgICAibWluWm9vbSI6IDAsCiAgICAgICAgIm5vV3JhcCI6IGZhbHNlLAogICAgICAgICJzdWJkb21haW5zIjogImFiYyIKfSkuYWRkVG8obWFwXzIzNjEyNDE5ZjEzMjQzNzRhMTk0NjQ4MTdhZTU2ZGJhKTsKICAgIAogICAgICAgICAgICB2YXIgZmVhdHVyZV9ncm91cF9hMWY3YzI1MjU4ZmU0N2IwOTU4YTUwZjcyYTJjMDZiMiA9IEwuZmVhdHVyZUdyb3VwKAogICAgICAgICAgICAgICAgKS5hZGRUbyhtYXBfMjM2MTI0MTlmMTMyNDM3NGExOTQ2NDgxN2FlNTZkYmEpOwogICAgICAgIAogICAgCiAgICAgICAgICAgICAgICB2YXIgcG9seV9saW5lXzAwZDY3NmVlMWVjYzRkZDBhOThkOTU4MDA2MzliYzExID0gTC5wb2x5bGluZSgKICAgICAgICAgICAgICAgICAgICBbW1s0Mi44Mjg3MjMsIDE0MS42NTA4ODFdLCBbNDIuODI4OTk4LCAxNDEuNjUwNThdLCBbNDIuODI4NjI4LCAxNDEuNjUwMDA2XSwgWzQyLjgyNzcyNiwgMTQxLjY1MTAzXSwgWzQyLjgyODY1MSwgMTQxLjY0OTk4NF0sIFs0Mi44MzQ

In [15]:
osm_map.save(outfile="map.html")